In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import folium
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        df= pd.read_csv(r'/kaggle/input/brooklyn-food-waste/brooklyn.csv')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Using data to map locations visited

Using Folium I created a map located the collection puts with some categorical information

In [ ]:
df.shape

In [ ]:
df.loc[df.duplicated()]

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes


In [ ]:
df['retailer_type'].unique()

## Data Set looks good for our purposes. 
### We will create a dataframe with the information we want to put on our map and use Folium to map it out

In [ ]:
test=df.groupby(['collection_lat',	'collection_long'	,'retailer_type',	'retailer_detail', ])['approximate_dollar_value'].sum()
test=test.reset_index()

In [ ]:
location_data = pd.DataFrame()
location_data['latitude']= test['collection_lat']
location_data['longitude']= test['collection_long']
location_data['retailer_type']=test['retailer_type']
location_data['dollar_value_total']=test['approximate_dollar_value']

In [ ]:
#create a map using latitude and longitude, respectively, of general Brooklyn through a google search
map=folium.Map(location=[40.6782,-73.99447], zoom_start=10)

#create a feature group to add to the map
dumpsters = folium.map.FeatureGroup()

#create loop to add location of dumpsters to the map
for lat, lon, in zip(location_data.latitude, location_data.longitude):
    dumpsters.add_child(
        folium.features.CircleMarker(
            [lat,lon],
            radius= 8,
            color= 'blue',
            fill= True,
            fill_color= 'red',
            fill_opacity=0.7
        )
    )
#adding some text to markers
latitudes = list(location_data.latitude)
longitudes = list(location_data.longitude)
store_label = location_data.retailer_type
dollar_label= location_data.dollar_value_total.round(2)

for lat, lon, label1, label2 in zip(latitudes, longitudes, store_label,dollar_label ):
    folium.Marker([lat, lon], popup=[label1, label2]).add_to(map)

#add features to map
map.add_child(dumpsters)